<a href="https://colab.research.google.com/github/TimStolp/iFlow/blob/master/iFlow_2D_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**2D Visualization**

In [15]:
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [16]:
%pip install tensorboardX
%pip install portalocker


## Train models for different seeds


In [22]:
seeds = [1, 2]
for seed in seeds:
  data_args = "1000_5_2_2_3_" + str(seed) + "_gauss_xtanh_u_f"
  # train 2d example (iVAE)
  !cd drive/MyDrive/iFlow ; python main.py --data-args=$data_args --epochs=20 --batch-size=8 --i-what=iVAE --cuda --preload-gpu
  # train 2d example (iFlow)
  !cd drive/MyDrive/iFlow ; python main.py --data-args=$data_args --epochs=20 --batch-size=8 --i-what=iFlow --cuda --flow_type=RQNSF_AG --nat_param_act=Softplus --preload-gpu

Namespace(anneal=False, batch_size=8, cuda=True, data_args='1000_5_2_2_3_1_gauss_xtanh_u_f', depth=3, epochs=20, file=None, flow_length=10, flow_type='RQSplineFlow', gpu_id='0', hidden_dim=50, i_what='iVAE', latent_dim=None, log_freq=25, lr=0.001, lr_drop_factor=0.5, lr_patience=10, max_iter=None, nat_param_act='Sigmoid', no_log=False, num_bins=8, preload=True, seed=1)
training on Tesla T4
data loaded on cuda
setup time: 4.1602091789245605s
Beginning training for exp: 1
.. checkpoint at iteration 0 ..
epoch 1: 4.5854s;	loss: 3.6888;	perf: 0.5713
epoch 2: 4.5489s;	loss: 0.9989;	perf: 0.6844
Epoch    60: reducing learning rate of group 0 to 5.0000e-04.
epoch 3: 4.6202s;	loss: 0.7901;	perf: 0.6735
Epoch    92: reducing learning rate of group 0 to 2.5000e-04.
epoch 4: 4.3203s;	loss: 0.7925;	perf: 0.6974
.. checkpoint at iteration 2500 ..
Epoch   117: reducing learning rate of group 0 to 1.2500e-04.
epoch 5: 4.4070s;	loss: 0.5299;	perf: 0.6731
Epoch   128: reducing learning rate of group 0 

# Create plots

In [2]:
def plot_2d(s, x, u, z_est_iFlow, z_est_iVAE):
  """
  s : true latent variables, source of the observations
  x : observations, the inputs of iFlow and iVAE models
  z_est_iFlow : predictions of the iFlow model
  z_est_iVAE : predictions of the iVAE model 
  """
  fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4)
  fig.tight_layout()

  n, n_segments = u.shape
  n_per_segment = int(n / n_segments)
  for i in range(n_segments):
    (start, stop) = i * n_per_segment, (i + 1) * n_per_segment
    ax1.scatter(s[start:stop,0], s[start:stop,1], s=1)
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_title("Original sources")
    ax2.scatter(x[start:stop,0], x[start:stop,1], s=1)
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_title("Observations")
    ax3.scatter(z_est_iFlow[start:stop,0], z_est_iFlow[start:stop,1], s=1)
    ax3.set_xticks([])
    ax3.set_yticks([])
    ax3.set_title("iFlow")
    ax4.scatter(z_est_iVAE[start:stop,0], z_est_iVAE[start:stop,1], s=1)
    ax4.set_xticks([])
    ax4.set_yticks([])
    ax4.set_title("iVAE")
  plt.show()
  return


In [1]:
for seed in seeds:
  # load data
  path_to_dataset = "drive/MyDrive/iFlow/data/1/"
  with np.load(path_to_dataset + "tcl_1000_5_2_2_3_" + str(seed) + "_gauss_xtanh_u.npz") as data:
    x = data['x']
    u = data['u']
    s = data['s']
    m = data['m']
    L = data['L']
  # load predictions
  path_to_z_est = "drive/MyDrive/iFlow/z_est/" + str(seed) + "/"
  z_est_iFlow = np.load(path_to_z_est + "z_est_iFlow.npy")
  z_est_iVAE = np.load(path_to_z_est + "z_est_iVAE.npy")
  # plot
  plot_2d(s, x, u, z_est_iFlow, z_est_iVAE)

NameError: ignored